In [0]:
%run ./01-config

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp, col, regexp_replace
import requests
import uuid

class Silver():
    def __init__(self):
        print("Initializing Spark Session...")
        self.spark = SparkSession.builder.appName("SilverIngestion").getOrCreate()
        app_name = self.spark.conf.get("spark.app.name", "SilverIngestion")
        print(f"🔹 Spark App Name: {app_name}")

        print("Loading configuration settings...")
        conf = Config()
        self.nyc_population_by_community_bronze_table = "nyc_population_by_community_raw"
        self.nyc_restaurant_inspection_bronze_table = "nyc_restaurant_inspection_raw"
        self.silver_path = conf.storage_account + "silver_db/"
        self.catalog_name = conf.catalog_name
        self.bronze_db_name = "bronze_db"
        self.db_name = "silver_db"
        print("✅ Configuration Loaded:")
        print(f"   - Storage Path for Silver Layer: {self.silver_path}")
        print(f"   - Catalog Name: {self.catalog_name}")
        print(f"   - Database Name: {self.db_name}")
        print("🚀 Silver Ingestion Initialized Successfully! 🎯\n")

    def table_exists(self, silver_table_name):
        """Check if a Delta table exists in the catalog."""
        try:
            self.spark.table(silver_table_name)
            return True
        except:
            return False
    
    def dim_borough_population(self):
        """Load NYC Borough Population data into the Silver layer with incremental loading."""
        process_id = str(uuid.uuid4())
        print("🔹 Starting silver layer data ingestion for NYC Borough Population...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_population_by_community_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.dim_borough_population"
        dim_borough_population_storage_path = self.silver_path + "dim_borough_population/"

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")
            existing_df = self.spark.table(silver_table_name)
            print(f"🔍 Existing record count: {existing_df.count()}")

            df = spark.read.table(bronze_table_name)
            columns = df.columns
            population_cols = [col_name for col_name in df.columns if "population" in col_name]
            df_unpivoted = df.melt(["borough", "cd_name"], population_cols, "year", "population")
            final_df = df_unpivoted.withColumn("borough", col("borough").upper())\
                                  .withColumn("cd_name", col("cd_name").upper())\
                                  .withColumn("year", regexp_replace(col("year"), "^_|_population", ""))\
                                  .withColumn("year", col("year").cast("int"))\
                                  .withColumn("population", col("population").cast("int"))\
                                  .withColumn("process_id", lit(process_id))\
                                  .withColumn("source", lit(bronze_table_name))\
                                  .withColumn("ingestion_timestamp", current_timestamp())
            new_df = final_df.join(existing_df,["borough", "cd_name", "year"], "left_anti")
            
            new_records_count = max(new_df.count() - existing_df.count(), 0)
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                df.write.format("delta").mode("append").saveAsTable(silver_table_name)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            self.spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name}(
                               borough STRING,
                               cd_name STRING,
                               year INT,
                               population INT,
                               process_id STRING,
                               source STRING,
                               ingestion_timestamp TIMESTAMP)
                            USING DELTA
                            LOCATION '{dim_borough_population_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            df = spark.read.table(bronze_table_name)

            columns = df.columns

            population_cols = [col_name for col_name in df.columns if "population" in col_name]

            df_unpivoted = df.melt(["borough", "cd_name"], population_cols, "year", "population")

            final_df = df_unpivoted.withColumn("borough", col("borough").upper())\
                                  .withColumn("cd_name", col("cd_name").upper())\
                                  .withColumn("year", regexp_replace(col("year"), "^_|_population", ""))\
                                  .withColumn("year", col("year").cast("int"))\
                                  .withColumn("population", col("population").cast("int"))\
                                  .withColumn("process_id", lit(process_id))\
                                  .withColumn("source", lit(bronze_table_name))\
                                  .withColumn("ingestion_timestamp", current_timestamp())

            final_df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {final_df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")


    def dim_restaurant(self):
        """Load NYC Restaurant data into the Silver layer with incremental loading."""
        process_id = str(uuid.uuid4())
        print("🔹 Starting silver layer data ingestion for NYC Restaurant data...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_restaurant_inspection_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.dim_restaurant"
        dim_restaurant_storage_path = self.silver_path + "dim_restaurant/"
        base_query = self.spark.sql(f"""
                                    SELECT DISTINCT
                                        CAST(CAMIS AS INT) AS restaurant_id,
                                        UPPER(DBA) AS restaurant_name,
                                        NVL(UPPER(CUISINE_DESCRIPTION), 'NOT AVAILABLE') AS cuisine_type,
                                        NVL(PHONE, 'NOT AVAILABLE') AS phone_no,
                                        BUILDING AS building_no, 
                                        STREET AS street_name,
                                        CASE WHEN UPPER(BORO) = '0' THEN 'NOT AVAILABLE' ELSE UPPER(BORO) END AS borough,
                                        ZIPCODE AS zipcode,
                                        CAST(longitude AS DOUBLE) AS longitude,
                                        CAST(latitude AS DOUBLE) AS latitude
                                    FROM {bronze_table_name}
                                """)

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")

            df = base_query.withColumn("process_id", lit(process_id))\
                        .withColumn("source", lit(bronze_table_name))\
                        .withColumn("ingestion_timestamp", current_timestamp())

            df.createOrReplaceTempView("temp_view")

            existing_df = self.spark.table(silver_table_name)
            existing_df = existing_df.drop("process_id", "source", "ingestion_timestamp", "start_date", "end_date")
            print(f"🔍 Existing record count: {existing_df.count()}")
            new_df = base_query.exceptAll(existing_df)
            new_records_count = new_df.count()
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                self.spark.sql(f"""
                                MERGE INTO {silver_table_name} AS TARGET
                                USING temp_view AS SOURCE
                                ON TARGET.restaurant_id = SOURCE.restaurant_id AND TARGET.END_DATE IS NULL
                                WHEN MATCHED AND TARGET.INGESTION_TIMESTAMP < SOURCE.INGESTION_TIMESTAMP THEN 
                                UPDATE SET TARGET.END_DATE = CURRENT_TIMESTAMP()
                                WHEN NOT MATCHED THEN 
                                INSERT (
                                    restaurant_id, restaurant_name, cuisine_type, phone_no, building_no, 
                                    street_name, borough, zipcode, longitude, latitude, process_id, 
                                    source, ingestion_timestamp, start_date, end_date
                                ) 
                                VALUES (
                                    SOURCE.restaurant_id, SOURCE.restaurant_name, SOURCE.cuisine_type, SOURCE.phone_no, 
                                    SOURCE.building_no, SOURCE.street_name, SOURCE.borough, SOURCE.zipcode, 
                                    SOURCE.longitude, SOURCE.latitude, '{process_id}', '{bronze_table_name}', 
                                    current_timestamp(), current_timestamp(), null
                                )
                            """)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            self.spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name} (
                                restaurant_id INT,
                                restaurant_name STRING,
                                cuisine_type STRING,
                                phone_no STRING,
                                building_no STRING,
                                street_name STRING,
                                borough STRING,
                                zipcode STRING,
                                longitude DOUBLE,
                                latitude DOUBLE,
                                process_id STRING,
                                source STRING,
                                ingestion_timestamp TIMESTAMP,
                                start_date TIMESTAMP,
                                end_date TIMESTAMP)
                            USING DELTA
                            LOCATION '{dim_restaurant_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            df = base_query.withColumn("process_id", lit(process_id))\
                            .withColumn("source", lit(bronze_table_name))\
                            .withColumn("ingestion_timestamp", current_timestamp())\
                            .withColumn("start_date", current_timestamp())\
                            .withColumn("end_date", lit(None).cast("timestamp"))

            df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")

In [0]:
silver = Silver()

In [0]:
silver.dim_restaurant()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS anubhav_data_hackathon.silver_db.dim_restaurant (
    restaurant_id INT,
    restaurant_name STRING,
    restaurant_location STRING,
    restaurant_building_no STRING,
    restaurant_street_name STRING,
    restaurant_zipcode STRING,
    restaurant_phone STRING,
    cuisine_type STRING
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS anubhav_data_hackathon.silver_db.dim_voilation (
    violation_code STRING,
    critical_flag STRING,
    violation_description STRING
);


In [0]:
%sql
INSERT OVERWRITE anubhav_data_hackathon.silver_db.dim_voilation (
    violation_code,
    critical_flag,
    violation_description
)
SELECT DISTINCT VIOLATION_CODE, CRITICAL_FLAG, MAX(VIOLATION_DESCRIPTION) AS VIOLATION_DESCRIPTION
FROM anubhav_data_hackathon.bronze_db.nyc_restaurant_inspection_raw
WHERE VIOLATION_CODE IS NOT NULL
GROUP BY VIOLATION_CODE, CRITICAL_FLAG;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS anubhav_data_hackathon.silver_db.fact_restaurant_inspections (
    restaurant_id STRING,
    violation_code STRING,
    inspection_date DATE,
    inspection_type STRING,
    action STRING,
    score STRING,
    grade STRING,
    grade_date DATE,
    record_date TIMESTAMP
) USING DELTA;


In [0]:
%sql
INSERT OVERWRITE anubhav_data_hackathon.silver_db.fact_restaurant_inspections
SELECT 
    a12.restaurant_id,
    COALESCE(a13.violation_code, 'No Violations') AS violation_code, 
    a11.inspection_date,
    COALESCE(a11.inspection_type, 'None') AS inspection_type,
    COALESCE(a11.action, 'None') AS action, 
    COALESCE(a11.SCORE, 'Not Available') AS SCORE, 
    COALESCE(a11.GRADE, 'Not Available') AS GRADE,
    CAST(NULLIF(a11.GRADE_DATE, 'Not Available') AS DATE) AS GRADE_DATE,
    a11.RECORD_DATE
FROM anubhav_data_hackathon.bronze_db.nyc_restaurant_inspection_raw a11
JOIN anubhav_data_hackathon.silver_db.dim_restaurant a12
  ON a11.camis = a12.restaurant_id
LEFT JOIN anubhav_data_hackathon.silver_db.dim_voilation a13
  ON a11.violation_code = a13.violation_code;